# TD : Les bases de pandas

Le but de ce TD est de vous familiariser avec le package pandas pour l'analyse de données et matplotlib pour la visualisation de données.

Nous allons travailler avec un fichier contenant les données d'un jour de ventes dans une chaîne de restaurants de restauration rapide (Chipotle).

Commençons par importer le package pandas :

In [3]:
import pandas as pd

Puis importons les données dans un DataFrame pandas :

In [4]:
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'
df = pd.read_csv(url, sep = '\t') # les données sont séparées par un tab

## Tâche 1 : comprendre les données / nettoyage

Pour commencer, essayons de comprendre la structure des données.

In [12]:
# Afficher les 7 premières lignes du dataframe
df.head(7)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69


Les données contiennent les informations suivantes :

*   `order_id` : l'indice de la commande
*   `quantity` : le nombre de produits commandés
*   `item_name` : le nom du produit commandé
*   `choice_description` : le détail des ingrédients en option
*   `item_price` : le prix total de la commande

Ainsi, à une même vente est associé un unique `order_id`, et donc (potentiellement) plusieurs lignes du dataframe selon si le client correspondant a commandé un ou plusieurs items différents du menu.


In [15]:
# Quel est le nombre de lignes dans ce jeu de données ?
df.shape[0]

4622

In [16]:
# Quel est le nombre de colonnes ?
df.shape[1]

5

In [17]:
# Vérifier s'il y a des données manquantes
df.isnull().any()

order_id              False
quantity              False
item_name             False
choice_description     True
item_price            False
dtype: bool

In [18]:
# Afficher le nom de toutes les colonnes
df.columns

Index(['order_id', 'quantity', 'item_name', 'choice_description',
       'item_price'],
      dtype='object')

In [21]:
# Afficher le type de toutes les colonnes
df.dtypes

# Quel est le type de la 5ème colonne ?
type(df.iloc[0, 4])

str

La dernière colonne (le prix) est une chaîne de caractères commençant par '\$'. Transformons-la en float. Pour cela, vérifions tout d'abord que tous les éléments commencent bien par '\$'.

Indice : utiliser le package numpy et la fonction np.all() qui vérifie que tous les éléments d'une liste sont True.

In [26]:
import numpy as np
np.all([x.startswith('$') for x in df.item_price])

True

In [33]:
def dollar_to_float(s):
  return float(s[1:])

# # df.item_price = [float(s[1:]) for s in df.item_price]
df.item_price = df.item_price.apply(dollar_to_float)
df.head()

2.34

## Tâche 2 : quelques analyses descriptives

Combien d'items différents le menu contient-il ?

In [34]:
len(set(df.item_name))

50

Quel est le prix moyen d'une vente (définie par un unique `order_id`) ? Le nombre moyen d'items par vente ? Attention, il faut commencer par regrouper les ventes par `order_id`.

In [35]:
df.groupby('order_id').sum().item_price.mean()

<ipython-input-35-c8ec9d5b5162>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('order_id').sum().item_price.mean()


18.811428571428568

Quel est l'intervalle des prix de vente ? (vente la moins chère et vente la plus chère)

In [37]:
print(df.groupby('order_id').sum().item_price.min(), df.groupby('order_id').sum().item_price.max())

10.08 205.25


<ipython-input-37-05e452be75c4>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print(df.groupby('order_id').sum().item_price.min(), df.groupby('order_id').sum().item_price.max())
<ipython-input-37-05e452be75c4>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print(df.groupby('order_id').sum().item_price.min(), df.groupby('order_id').sum().item_price.max())


Combien de ventes différentes ont-elles été effectuées ce jour-là ?

In [39]:
len(np.unique(df.order_id))

1834

Quel item a été le plus vendu ? Indice : la méthode DataFrame.idxmax() renvoie la ligne du DataFrame correspondant au maximum de la colonne.

Quel nombre de cet item a-t-il été vendu ?

Optionnel (sautez la question et revenez-y plus tard si le temps le permet) : Quel a été l'item le plus vendu dans la colonne "choice_description" ?

La colonne prix correspond au prix total payé pour chaque ligne. Calculer le prix unitaire de chaque item dans chaque commande.

En fonction des options choisies (spécifiées dans `choice_description`), le pprix du même item peut varier. Quel est donc l'intervalle de prix pour chaque item du menu ?

Calculer combien d'items coûtent toujous plus que $8.

Calculer le pourcentage des ventes dû à chaque item du menu.

 À quel pourcentage des ventes correspondent les 5 items les plus vendus en nombre ? Les 5 items dont le pourcentage des ventes est le plus haut ?

Optionnel : quel est le choix le plus populaire comme accompagnement de "Chicken Bowl" ? Noter que la colonne "choice_description" est de type string.

Indices : si x est de type string, x.replace('c', 'd') remplace toutes les occurences de 'c' par 'd' dans x. x.split('e') divise x selon les occurrences de 'e' dans x en une liste de strings.


Comptons à présent le nombre de fois que chaque accompagnement de "Chicken Bowl" a été choisi.

Pour terminer, calculons le nombre moyen d'accompagnements par burrito.

## Tâche 3 : visualisation des données

Testons maintenant ce qu'on a appris sur matplotlib pour effectuer quelques visualisations simples du jeu de données.

Commençons par afficher la distribution des valeurs de ventes. Jouer par exemple sur le paramètre `bins` (si entier = nombre de barres d'histogrammes, si liste = intervalles des barres).

Il est difficile de bien visualiser la distribution des données. Passons donc en axes logarithmiques. En matplotlib, cela se fait simplement avec la commande `plt.yscale('log')` (de même pour `xscale`), ou bien `ax.set_yscale('log')` si on travaille avec un axe.

Affichons maintenant deux histogrammes côte à côte : celui des valeurs de ventes que nous venons de calculer, et celui des prix moyens de chaque item du menu.

Affichons à présent le nombre de ventes par "item_name" pour les 10 items les plus vendus. Indice : utiliser la fonction `plt.bar()` pour afficher un barplot, qui prend en arguments `x` (les abscisses des rectangles) et `height` (leur hauteur).

Affichons maintenant les noms des items sur l'axe des abscisses. Nous allons utiliser la fonction `plt.xticks(ticks=..., labels=..., rotation=...)` où `ticks` correspond aux abscisses de nos points de données, `labels` à la liste des labels désirés et `rotation` l'angle de rotation.